# 1.数据导入与划分

In [1]:
from pyspark.sql import SparkSession # SparkSession 是Spark 2.0版本的新入口
spark = SparkSession.builder.master('local').getOrCreate()

In [3]:
raw_data = spark.read.csv(path="hdfs://localhost:9000/user/bdlab/lab2/SUSY.csv.gz",header=False,inferSchema=True)

In [9]:
raw_data.printSchema()

root
 |-- _c0: double (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: double (nullable = true)
 |-- _c9: double (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: double (nullable = true)
 |-- _c12: double (nullable = true)
 |-- _c13: double (nullable = true)
 |-- _c14: double (nullable = true)
 |-- _c15: double (nullable = true)
 |-- _c16: double (nullable = true)
 |-- _c17: double (nullable = true)
 |-- _c18: double (nullable = true)



In [29]:
raw_data.count()

5000000

In [4]:
train,test = raw_data.randomSplit([0.8,0.2])

In [45]:
raw_data.groupBy(raw_data[0]).count().collect()

[Row(_c0=0.0, count=2712173), Row(_c0=1.0, count=2287827)]

In [86]:
prob_0 = 2712173/5000000
prob_1 = 1-prob_0

In [5]:
train.groupBy(raw_data[0]).count().collect()

[Row(_c0=0.0, count=2170505), Row(_c0=1.0, count=1829609)]

In [6]:
test.groupBy(raw_data[0]).count().collect()

[Row(_c0=0.0, count=541668), Row(_c0=1.0, count=458218)]

In [ ]:
# 划分后写入hdfs
test.write.csv(path="hdfs://localhost:9000/user/bdlab/lab2/SUSY_test.csv.gzip",compression="gzip")
train.write.csv(path="hdfs://localhost:9000/user/bdlab/lab2/SUSY_train.csv.gzip",compression="gzip")

测试集处理

In [141]:
test_data = test.select(test.columns[1:])

In [142]:
test_label = test.select(test.columns[0])

# 2.Naive-Bayes

In [148]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

In [12]:
# 加载train
# train = spark.read.csv(path="hdfs://localhost:9000/user/bdlab/lab2/SUSY_train.csv.gzip",header=False,inferSchema=True)

In [74]:
cal_mean_func = { _:"mean" for _ in train.columns[1:] }
cal_var_func = { _:"variance" for _ in train.columns[1:]}

In [131]:
mean_train = train.groupBy(train[0]).agg(cal_mean_func).collect()
var_train = sample_train.groupBy(train[0]).agg(cal_var_func).collect()

CPU times: user 23.8 ms, sys: 7.06 ms, total: 30.9 ms
Wall time: 1min 57s


In [143]:
mean_0 = [ mean_train[0]["avg(_c"+str(i)+str(")")] for i in range(1,19)]
mean_1 = [ mean_train[1]["avg(_c"+str(i)+str(")")] for i in range(1,19)]
var_0 = [ var_train[0]["variance(_c"+str(i)+str(")")] for i in range(1,19)]
var_1 = [ var_train[0]["variance(_c"+str(i)+str(")")] for i in range(1,19)]

In [135]:
const_pi = 1/ np.sqrt(2*np.pi)
def GaussianProb(x,miu,sigmaq):
    left = const_pi / np.sqrt(sigmaq)
    right = np.exp(- (x - miu)**2 / (2 * sigmaq))
    return left*right

def NB_classifier(x):
    """
    input:
        x: a vec with len of 18
    ouput:
        result: 0 or 1
    """
    if len(x) != 18:
        raise ValueError
        
    # 计算 0 概率
    result_0 = prob_0
    for i in range(18):
        result_0 = result_0 * GaussianProb(x[i],mean_0[i],var_0[i])
        
    # 计算 1 概率
    result_1 = prob_1
    for i in range(18):
        result_1 = result_1 * GaussianProb(x[i],mean_1[i],var_1[i])
    return 0 if result_0>result_1 else 1

def NB_predict(testset):
    return [ NB_classifier(_) for _ in testset ]

In [156]:
np.save("test_data_arr",test_data_arr)

In [160]:
np.save("test_label_arr",test_label_arr)

# 3.Logistic-Regression

In [165]:
def sigmod(x,theta):
    return 1/(1+np.exp( - np.matmul(x,theta)))